In [1]:
## imports
import pandas as pd
import numpy as np
import plotnine
from plotnine import *
import random

## print multiple things from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## Load data

In [3]:
## load data on 2020 crimes in DC
dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")
dc_crim_2020.head()
dc_crim_2020.shape
dc_crim_2020.info()


,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID
0,-76.936390,38.887070,20059143,2020/04/15 00:34:11+00,MIDNIGHT,OTHERS,THEFT F/AUTO,4555 - 4612 BLOCK OF BENNING ROAD SE,405519.0,135471.0,...,009906 1,9906.0,Precinct 104,38.887062,-76.936387,NaN,2020/04/13 22:00:24+00,2020/04/14 10:00:32+00,587518163,20059143-01
1,-76.932191,38.902985,20059150,2020/04/15 01:00:10+00,MIDNIGHT,OTHERS,MOTOR VEHICLE THEFT,4816 - 4899 BLOCK OF SHERIFF ROAD NE,405882.0,137238.0,...,007809 2,7809.0,Precinct 94,38.902977,-76.932188,NaN,2020/04/13 17:00:37+00,2020/04/14 10:00:45+00,587518164,20059150-01
2,-76.994989,38.881997,20059157,2020/04/15 01:51:11+00,MIDNIGHT,OTHERS,BURGLARY,500 - 699 BLOCK OF 8TH STREET SE,400435.0,134906.0,...,007000 2,7000.0,Precinct 90,38.881989,-76.994987,CAPITOL HILL,2020/04/14 23:57:49+00,2020/04/15 01:51:19+00,587518165,20059157-01
3,-77.050128,38.901346,20059162,2020/04/15 01:53:05+00,MIDNIGHT,OTHERS,THEFT F/AUTO,900 - 999 BLOCK OF 23RD STREET NW,395652.0,137055.0,...,005600 4,5600.0,Precinct 3,38.901338,-77.050126,NaN,2020/03/13 09:00:22+00,2020/03/19 13:00:37+00,587518166,20059162-01
4,-76.973098,38.861905,20059168,2020/04/15 02:16:13+00,MIDNIGHT,OTHERS,THEFT/OTHER,2301 - 2399 BLOCK OF GOOD HOPE COURT SE,402335.0,132676.0,...,007502 1,7502.0,Precinct 134,38.861898,-76.973096,NaN,2020/04/14 17:00:28+00,2020/04/14 22:05:40+00,587518167,20059168-01


(27875, 25)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27875 entries, 0 to 27874
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   X                     27875 non-null  float64
 1   Y                     27875 non-null  float64
 2   CCN                   27875 non-null  int64  
 3   REPORT_DAT            27875 non-null  object 
 4   SHIFT                 27875 non-null  object 
 5   METHOD                27875 non-null  object 
 6   OFFENSE               27875 non-null  object 
 7   BLOCK                 27875 non-null  object 
 8   XBLOCK                27875 non-null  float64
 9   YBLOCK                27875 non-null  float64
 10  WARD                  27875 non-null  int64  
 11  ANC                   27875 non-null  object 
 12  DISTRICT              27874 non-null  float64
 13  PSA                   27874 non-null  float64
 14  NEIGHBORHOOD_CLUSTER  27865 non-null  object 
 15  BLOCK_GROUP        

## Practice with aggregation 

### Example of grouping by one variable and doing one aggregation 

**Task**: find the number of unique offense types by ward

In [21]:
## way one: use built in unique function
unique_off_byward = dc_crim_2020.groupby('WARD').agg({'OFFENSE': 'nunique'}).reset_index()
unique_off_byward

## way two: use lambda function
unique_off_byward_v2 = dc_crim_2020.groupby('WARD').agg({'OFFENSE': lambda x: len(x.unique())}).reset_index()
unique_off_byward_v2

,WARD,OFFENSE
0,1,9
1,2,9
2,3,8
3,4,8
4,5,9
5,6,9
6,7,9
7,8,9


,WARD,OFFENSE
0,1,9
1,2,9
2,3,8
3,4,8
4,5,9
5,6,9
6,7,9
7,8,9


### Example of grouping by one variable and providing two summaries of the same variable

**Task**: want to find out what offenses are in each ward; repeat the above but add a second variable that pastes the unique offenses in that ward separated by ; (e.g., Theft; Burglary;...)



In [9]:
## one way to solve -- write out the whole function inside the agg and use default varnames
unique_off_valuecount_byward = dc_crim_2020.groupby('WARD').agg({'OFFENSE': ['nunique',
                                            lambda x: "; ".join(sorted(x.unique()))]}).reset_index()

unique_off_valuecount_byward


## a second way to solve --- write the function outside of the agg and then feed that function
## to the agg (still uses lambda)
def find_join_unique(x):
    
    sorted_un = sorted(x.unique())
    joined_un = "; ".join(sorted_un)
    return(joined_un)

unique_off_valuecount_byward_v2 = dc_crim_2020.groupby('WARD').agg({'OFFENSE': ['nunique',
                                            lambda x: find_join_unique(x)]}).reset_index()


## a third way to solve --- give more informative column names by using a diff structure
## inside agg
unique_off_valuecount_byward_bettername = dc_crim_2020.groupby("WARD").agg(count_offense =('OFFENSE', 
                                                            lambda x: len(x.unique())), 
                                                            name_offense =('OFFENSE', 
                                                            lambda x: find_join_unique(x))).reset_index()

unique_off_valuecount_byward_bettername

WARD OFFENSE                                                   
       nunique                                         <lambda_0>
0    1       9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
1    2       9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
2    3       8  ASSAULT W/DANGEROUS WEAPON; BURGLARY; HOMICIDE...
3    4       8  ASSAULT W/DANGEROUS WEAPON; BURGLARY; HOMICIDE...
4    5       9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
5    6       9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
6    7       9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
7    8       9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...

,WARD,count_offense,name_offense
0,1,9,ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
1,2,9,ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
2,3,8,ASSAULT W/DANGEROUS WEAPON; BURGLARY; HOMICIDE...
3,4,8,ASSAULT W/DANGEROUS WEAPON; BURGLARY; HOMICIDE...
4,5,9,ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
5,6,9,ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
6,7,9,ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
7,8,9,ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...


### Example of grouping by two variables 

**Task**: group by ward and shift and find the offense that is most common in that ward and shift

In [10]:
## way one
top_w_shift = dc_crim_2020.groupby(['WARD', 'SHIFT']).agg({'OFFENSE': lambda x: x.value_counts(sort = True, 
                                                                    ascending = False).index[0]}).reset_index()
top_w_shift

,WARD,SHIFT,OFFENSE
0,1,DAY,THEFT/OTHER
1,1,EVENING,THEFT/OTHER
2,1,MIDNIGHT,THEFT/OTHER
3,2,DAY,THEFT/OTHER
4,2,EVENING,THEFT/OTHER
5,2,MIDNIGHT,THEFT/OTHER
6,3,DAY,THEFT/OTHER
7,3,EVENING,THEFT/OTHER
8,3,MIDNIGHT,THEFT/OTHER
9,4,DAY,THEFT F/AUTO


In [11]:
## way two: function defined outside the pandas dataframe (preview for next section)

def most_common(one_col: pd.Series):
    
    ## sort values
    sorted_series = one_col.value_counts(sort = True, ascending = False)
    
    ## get top 
    top = sorted_series.index[0]
    
    ## return
    return(top)

In [12]:
top_w_shift_alternate = dc_crim_2020.groupby(['WARD', 'SHIFT']).agg({'OFFENSE': lambda x: most_common(x)}).reset_index()

In [13]:

top_w_shift_alternate

,WARD,SHIFT,OFFENSE
0,1,DAY,THEFT/OTHER
1,1,EVENING,THEFT/OTHER
2,1,MIDNIGHT,THEFT/OTHER
3,2,DAY,THEFT/OTHER
4,2,EVENING,THEFT/OTHER
5,2,MIDNIGHT,THEFT/OTHER
6,3,DAY,THEFT/OTHER
7,3,EVENING,THEFT/OTHER
8,3,MIDNIGHT,THEFT/OTHER
9,4,DAY,THEFT F/AUTO


## Practice with recoding 

### Simple np.where

**Task**: create an indicator `is_theft` for any offense that contains theft
    
**Task**: create an indicator `is_theft_notmotor` for any offense that contains theft but does not contain the word motor

In [16]:
dc_crim_2020.OFFENSE.value_counts()

dc_crim_2020['is_theft'] = np.where(dc_crim_2020.OFFENSE.str.contains("THEFT"), True, False)

pd.crosstab(dc_crim_2020.is_theft, dc_crim_2020.OFFENSE)


dc_crim_2020['is_theft_notmotor'] = np.where(dc_crim_2020.OFFENSE.str.contains("THEFT") & 
                                            ~dc_crim_2020.OFFENSE.str.contains("MOTOR"), True, False)


pd.crosstab(dc_crim_2020.is_theft_notmotor, dc_crim_2020.OFFENSE)

THEFT/OTHER                   10904
THEFT F/AUTO                   8267
MOTOR VEHICLE THEFT            3258
ROBBERY                        1995
ASSAULT W/DANGEROUS WEAPON     1623
BURGLARY                       1445
HOMICIDE                        197
SEX ABUSE                       173
ARSON                            13
Name: OFFENSE, dtype: int64

OFFENSE,ARSON,ASSAULT W/DANGEROUS WEAPON,BURGLARY,HOMICIDE,MOTOR VEHICLE THEFT,ROBBERY,SEX ABUSE,THEFT F/AUTO,THEFT/OTHER
is_theft,,,,,,,,,
False,13,1623,1445,197,0,1995,173,0,0
True,0,0,0,0,3258,0,0,8267,10904


OFFENSE,ARSON,ASSAULT W/DANGEROUS WEAPON,BURGLARY,HOMICIDE,MOTOR VEHICLE THEFT,ROBBERY,SEX ABUSE,THEFT F/AUTO,THEFT/OTHER
is_theft_notmotor,,,,,,,,,
False,13,1623,1445,197,3258,1995,173,0,0
True,0,0,0,0,0,0,0,8267,10904


### Row filtering 

**Task**: filter to crime reports about theft using the `is_theft` definition and that are in ward 3
    

In [17]:
theft_w3 = dc_crim_2020[(dc_crim_2020.is_theft) &
                       (dc_crim_2020.WARD == 3)].copy()

theft_w3.head()

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,is_theft,is_theft_notmotor
52,-77.051908,38.924199,20059618,2020/04/16 08:03:43+00,DAY,OTHERS,THEFT F/AUTO,2600 - 2649 BLOCK OF CONNECTICUT AVENUE NW,395499.0,139592.0,...,Precinct 136,38.924191,-77.051906,NaN,2020/04/14 09:00:21+00,2020/04/14 09:30:28+00,587518215,20059618-01,True,True
83,-77.086974,38.959040,20059783,2020/04/16 17:22:42+00,EVENING,OTHERS,MOTOR VEHICLE THEFT,4400 - 4499 BLOCK OF JENIFER STREET NW,392462.0,143462.0,...,Precinct 31,38.959032,-77.086972,NaN,2020/03/27 21:55:01+00,2020/03/27 22:05:22+00,587518246,20059783-01,True,False
117,-77.077705,38.944372,20057050,2020/04/09 19:05:43+00,EVENING,OTHERS,THEFT/OTHER,4227 - 4299 BLOCK OF WISCONSIN AVENUE NW,393264.0,141833.0,...,Precinct 30,38.944364,-77.077702,NaN,2020/04/09 18:36:34+00,1970/01/01 00:00:00+00,587518280,20057050-01,True,True
123,-77.063920,38.944876,20057118,2020/04/10 02:07:24+00,MIDNIGHT,OTHERS,THEFT/OTHER,4200 - 4399 BLOCK OF CONNECTICUT AVENUE NW,394459.0,141888.0,...,Precinct 34,38.944868,-77.063918,NaN,2020/04/09 21:11:43+00,2020/04/09 21:20:18+00,587518286,20057118-01,True,True
128,-77.055500,38.930368,20057152,2020/04/09 23:14:59+00,MIDNIGHT,OTHERS,THEFT/OTHER,3000 - 3199 BLOCK OF CONNECTICUT AVENUE NW,395188.0,140277.0,...,Precinct 26,38.930360,-77.055497,NaN,2020/04/09 22:32:21+00,1970/01/01 00:00:00+00,587518291,20057152-01,True,True


### Np.select

**Task**: create a new variable, `offense_summary`, where you:
        
- Recode theft offenses that use a gun or knife as violent theft
- Recode non-theft offenses that use a gun or knife as violent other
- Recode all other as non-violent 

In [18]:
dc_crim_2020.METHOD.value_counts()

crime_criteria = [dc_crim_2020.METHOD.str.contains("GUN|KNIFE") &
                 dc_crim_2020.OFFENSE.str.contains('THEFT'),
                 dc_crim_2020.METHOD.str.contains("GUN|KNIFE") &
                 ~dc_crim_2020.OFFENSE.str.contains('THEFT')]


OTHERS    25239
GUN        2013
KNIFE       623
Name: METHOD, dtype: int64

In [19]:
## first, create a list of conditions
df = dc_crim_2020.copy()
conditions_offense = [df['is_theft'] & df['METHOD'].isin(['GUN', 'KNIFE']),
                     ~df['is_theft'] & df['METHOD'].isin(['GUN', 'KNIFE'])]

code_to = ['violent theft', 'violent other']

## create new col
df['offense_summary'] = np.select(conditions_offense,
                                 code_to, 
                                 default = 'non-violent')

df.offense_summary.value_counts()

## check
df.loc[(df.is_theft) &
  (df.METHOD.isin(['GUN', 'KNIFE'])),
['OFFENSE', 'offense_summary', 'METHOD']].head()


non-violent      25239
violent other     2632
violent theft        4
Name: offense_summary, dtype: int64

,OFFENSE,offense_summary,METHOD
2950,THEFT/OTHER,violent theft,GUN
9823,MOTOR VEHICLE THEFT,violent theft,GUN
13763,MOTOR VEHICLE THEFT,violent theft,GUN
19481,MOTOR VEHICLE THEFT,violent theft,GUN


### Map.recode

**Task**: recode shifts that are MIDNIGHT or EVENING as "nighttime"; code other shift to daytime

In [20]:
shifts_dict = {'MIDNIGHT': 'nighttime',
              'EVENING': 'nighttime'}

dc_crim_2020['summary_shifts'] = dc_crim_2020.SHIFT.map(shifts_dict).fillna("daytime")

pd.crosstab(dc_crim_2020.summary_shifts,
           dc_crim_2020.SHIFT)

SHIFT,DAY,EVENING,MIDNIGHT
summary_shifts,,,
daytime,10082,0,0
nighttime,0,12299,5494


## Loops v functions

In [95]:
## some preprocessing
dc_crim_2020['report_dt'] = pd.to_datetime(dc_crim_2020.REPORT_DAT)
dc_crim_2020[['report_dt', 'REPORT_DAT']].head()



,report_dt,REPORT_DAT
0,2020-12-22 17:58:29+00:00,2020/12/22 17:58:29+00
1,2020-12-18 21:04:23+00:00,2020/12/18 21:04:23+00
2,2020-07-08 22:13:24+00:00,2020/07/08 22:13:24+00
3,2020-07-24 12:42:24+00:00,2020/07/24 12:42:24+00
4,2020-10-26 22:26:55+00:00,2020/10/26 22:26:55+00


In [96]:
## moter vehicle theft
CCN_examples = ['20165648', '20123250']

## view those crimes
crimes_lookfor = dc_crim_2020.loc[dc_crim_2020.CCN.astype(str).isin(CCN_examples),
                ['CCN', 'WARD', 'OFFENSE', 'report_dt']].copy()

crimes_lookfor


,CCN,WARD,OFFENSE,report_dt
13880,20165648,6,MOTOR VEHICLE THEFT,2020-11-19 21:25:50+00:00
18533,20123250,2,MOTOR VEHICLE THEFT,2020-08-29 01:00:25+00:00


### Loop approach

In [98]:
from datetime import datetime, timedelta

In [113]:
## define crimes to look for and crimes to look within
CCN_examples = ['20165648', '20123250']
crimes_lookfor = dc_crim_2020.loc[dc_crim_2020.CCN.astype(str).isin(CCN_examples),
                ['CCN', 'WARD', 'OFFENSE', 'report_dt']].copy()
other_crimes = dc_crim_2020[~dc_crim_2020.CCN.astype(str).isin(CCN_examples)].copy()


## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i in range(0, crimes_lookfor.shape[0]):
    
    ## extract row
    one_row = crimes_lookfor.iloc[i]
    
    ## first, subset to crimes in same ward
    same_wards = other_crimes[other_crimes.WARD == one_row.WARD].copy()
    print(same_wards.WARD.value_counts())
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    cutoff = one_row.report_dt +  timedelta(minutes=20)
    print(one_row.report_dt)
    print(cutoff)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= one_row.report_dt) & 
                                    (same_wards.report_dt <= cutoff)].copy()
    print(same_wards_sametime)
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

## bad part

6    4752
Name: WARD, dtype: int64
2020-11-19 21:25:50+00:00
2020-11-19 21:45:50+00:00
               X          Y       CCN              REPORT_DAT    SHIFT  \
13463 -77.023972  38.907912  20165497  2020/11/19 21:38:59+00  EVENING   

       METHOD              OFFENSE                               BLOCK  \
13463  OTHERS  MOTOR VEHICLE THEFT  1300 - 1399 BLOCK OF 9TH STREET NW   

         XBLOCK    YBLOCK  ...  BID              START_DATE  \
13463  397921.0  137783.0  ...  NaN  2020/11/19 14:45:25+00   

                     END_DATE   OBJECTID OCTO_RECORD_ID is_after_christmas  \
13463  2020/11/19 16:03:07+00  587047293    20165497-01              False   

       is_theft is_theft_notmotor  summary_shifts                 report_dt  
13463      True             False       nighttime 2020-11-19 21:38:59+00:00  

[1 rows x 30 columns]
2    4360
Name: WARD, dtype: int64
2020-08-29 01:00:25+00:00
2020-08-29 01:20:25+00:00
Empty DataFrame
Columns: [X, Y, CCN, REPORT_DAT, SHIFT, METHOD, O

,,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,is_after_christmas,is_theft,is_theft_notmotor,summary_shifts,report_dt
20165648,13463,-77.023972,38.907912,20165497,2020/11/19 21:38:59+00,EVENING,OTHERS,MOTOR VEHICLE THEFT,1300 - 1399 BLOCK OF 9TH STREET NW,397921.0,137783.0,...,NaN,2020/11/19 14:45:25+00,2020/11/19 16:03:07+00,587047293,20165497-01,False,True,False,nighttime,2020-11-19 21:38:59+00:00


### Function approach

In [136]:
def proximate_crimes(search_for: pd.DataFrame,
                    search_in: pd.DataFrame):
    
    ## first, subset to crimes in same ward
    same_wards = search_in[search_in.WARD == search_for.WARD].copy()
    print(same_wards.WARD.value_counts())
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ### substep: get time cutoff
    cutoff = search_for.report_dt +  timedelta(minutes=20)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= search_for.report_dt) & 
                                    (same_wards.report_dt <= cutoff)].copy()
    
    ## add focal match
    same_wards_sametime['focal_crime'] = search_for.CCN
    
    ## return
    return(same_wards_sametime)
    

In [138]:
one_match = proximate_crimes(search_for = crimes_lookfor.iloc[0],
                            search_in = other_crimes)
one_match

all_matches_list = [proximate_crimes(crimes_lookfor.iloc[i],
                               other_crimes) for i in range(0, crimes_lookfor.shape[0])]
all_matches = pd.concat(all_matches_list)
all_matches # happens to only be one match

6    4752
Name: WARD, dtype: int64


,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,is_after_christmas,is_theft,is_theft_notmotor,summary_shifts,report_dt,focal_crime
13463,-77.023972,38.907912,20165497,2020/11/19 21:38:59+00,EVENING,OTHERS,MOTOR VEHICLE THEFT,1300 - 1399 BLOCK OF 9TH STREET NW,397921.0,137783.0,...,2020/11/19 14:45:25+00,2020/11/19 16:03:07+00,587047293,20165497-01,False,True,False,nighttime,2020-11-19 21:38:59+00:00,20165648


6    4752
Name: WARD, dtype: int64
2    4360
Name: WARD, dtype: int64


,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,is_after_christmas,is_theft,is_theft_notmotor,summary_shifts,report_dt,focal_crime
13463,-77.023972,38.907912,20165497,2020/11/19 21:38:59+00,EVENING,OTHERS,MOTOR VEHICLE THEFT,1300 - 1399 BLOCK OF 9TH STREET NW,397921.0,137783.0,...,2020/11/19 14:45:25+00,2020/11/19 16:03:07+00,587047293,20165497-01,False,True,False,nighttime,2020-11-19 21:38:59+00:00,20165648
